In [3]:
import time
import json
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.common.exceptions import NoAlertPresentException


def dismiss_alert(driver, dismiss=False):
    try:
        alert = driver.switch_to.alert
        if dismiss:
            alert.dismiss()
        else:
            alert.accept()  # Use alert.accept() if you want to accept the alert.
        print("Alert was present and dismissed.")
    except NoAlertPresentException:
        print("No alert present.")

class YouTubePublisher:
    def __init__(self, driver, video_path, thumbnail_path, metadata_path, test=False):
        self.driver = driver
        self.video_path = video_path
        self.thumbnail_path = thumbnail_path
        self.metadata_path = metadata_path
        self.test = test
        self.load_metadata()
        
    def load_metadata(self):
        try:
            with open(self.metadata_path, 'r') as file:
                self.metadata = json.load(file)
            print('Metadata loaded successfully.')
        except Exception as e:
            print(f"An error occurred while loading the metadata: {e}")
            self.metadata = {}
    
    def upload_video(self):
        try:
            self.driver.get('https://www.youtube.com/upload')
            time.sleep(1)  # Wait for the page to load
            
            dismiss_alert(driver)
            
            time.sleep(10)
            
            # Upload the video file
            absolute_video_path = str(Path.cwd() / self.video_path)
            self.driver.find_element(By.XPATH, "//input[@type='file']").send_keys(absolute_video_path)
            print('Attached video {}'.format(self.video_path))
        except Exception as e:
            raise Exception(f"Failed to upload video: {e}")
    
    def wait_for_processing(self):
        try:
            expected_text = "Checks complete. No issues found."
            wait = WebDriverWait(self.driver, 1200)  # Adjust timeout as needed
            span_xpath = f"//span[contains(@class, 'progress-label') and contains(@class, 'style-scope') and contains(@class, 'ytcp-video-upload-progress') and contains(text(), '{expected_text}')]"
            wait.until(EC.presence_of_element_located((By.XPATH, span_xpath)))
            print('The expected text is present in the span element.')
        except TimeoutException:
            raise Exception("Processing time exceeded the limit.")
    
    def set_video_details(self):
        try:
            video_title_with_tags = self.metadata["title"] # + " " + " ".join("#" + tag for tag in self.metadata["tags"])
            title_input_xpath = "//div[@id='textbox'][@contenteditable='true']"
            title_input = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, title_input_xpath)))
            title_input.clear()
            title_input.send_keys(video_title_with_tags)
            print(f'The video title was set to "{video_title_with_tags}"')
            
            time.sleep(3)
            
            description_input_xpath = "//div[@id='textbox'][@contenteditable='true' and @aria-label='Tell viewers about your video (type @ to mention a channel)']"
            description_input = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, description_input_xpath)))
            description_input.clear()
            description_input.send_keys(self.metadata["long_description"])
            print(f'The video description was set to "{self.metadata["long_description"]}"')
        except Exception as e:
            raise Exception(f"Failed to set video details: {e}")
    
    def set_thumbnail(self):
        try:
            absolute_thumbnail_path = str(Path.cwd() / self.thumbnail_path)
            self.driver.find_element(By.XPATH, "//input[@id='file-loader']").send_keys(absolute_thumbnail_path)
            print('Attached thumbnail {}'.format(self.thumbnail_path))
        except NoSuchElementException:
            print('Thumbnail upload option not available.')
        except Exception as e:
            raise Exception(f"Failed to set thumbnail: {e}")
            
    def set_playlist(self):
        # Add video to playlist (if specified)
        playlist_name = "SimpleLife"
        try:
            dropdown_trigger = self.driver.find_element(By.CSS_SELECTOR, "ytcp-text-dropdown-trigger.dropdown")
            dropdown_trigger.click()
            print('Clicked on playlist dropdown')

            wait = WebDriverWait(self.driver, 10)
            option_xpath = f"//span[contains(@class, 'style-scope') and text()='{playlist_name}']"
            playlist_option = wait.until(EC.visibility_of_element_located((By.XPATH, option_xpath)))
            playlist_option.click()
            print(f'Selected playlist: {playlist_name}')
            
            wait = WebDriverWait(self.driver, 10)  # Adjust timeout as needed
            done_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "ytcp-button.done-button")))
            done_button.click()
            print('Clicked on the Done button.')

        except Exception as e:
            print(f"An error occurred during playlist selection: {e}")
            
    def set_not_for_kids(self):
        # Click the 'Not Made for Kids' radio button
        try:
            wait = WebDriverWait(self.driver, 10)  # Adjust timeout as needed
            not_made_for_kids_button = wait.until(EC.element_to_be_clickable((By.NAME, "VIDEO_MADE_FOR_KIDS_NOT_MFK")))
            not_made_for_kids_button.click()
            print('Clicked on the Not Made for Kids button.')
        except Exception as e:
            print(f"An error occurred when trying to click on the Not Made for Kids button: {e}")
            raise Exception(f"Failed to set Not Made for Kids: {e}")


    
    def set_tags_and_more(self):
        try:
            show_more_button = WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.ID, "toggle-button")))
            show_more_button.click()
            print('Clicked on the Show more button.')
            
            tags = self.metadata["tags"]
            tags_string = ', '.join(tags) + ','
            tags_input_xpath = "//input[@id='text-input' and @class='text-input style-scope ytcp-chip-bar' and @aria-label='Tags']"
            tags_input = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, tags_input_xpath)))
            tags_input.send_keys(tags_string)
            print(f'Tags entered: {tags_string}')
        except TimeoutException:
            raise Exception("Failed to click on the Show more button or set tags.")
    
    def set_visibility_and_publish(self):
        try:
            for _ in range(3):
                next_button = WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.ID, "next-button")))
                next_button.click()
                time.sleep(2)
            
            # Set the video's visibility and publish
            self.driver.find_element(By.NAME, 'PUBLIC').click()
            
            if self.test:
                user_input = input("Do you want to publish now? Type 'yes' to confirm: ").strip().lower()
                if user_input != 'yes':
                    print("Publishing cancelled by user.")
                    return

            publish_button = WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.ID, "done-button")))
            publish_button.click()
            print('Clicked on the Publish button.')
        except TimeoutException:
            raise Exception("Failed to set visibility or click on the Publish button.")
    
    def publish_video(self):
            max_attempts = 3
            for attempt in range(max_attempts):
                try:
                    self.upload_video()
                    # time.sleep(5)
                    self.wait_for_processing()
                    self.set_video_details()
                    time.sleep(3)
                    self.set_thumbnail()
                    time.sleep(3)
                    self.set_playlist()
                    time.sleep(3)
                    self.set_not_for_kids()
                    time.sleep(3)
                    self.set_tags_and_more()
                    time.sleep(3)
                    
                    self.set_visibility_and_publish()
                    print("Video published successfully.")
                    break  # Break out of the loop if publish is successful
                except Exception as e:
                    print(f"Attempt {attempt + 1} of {max_attempts} failed: {e}")
                    if attempt < max_attempts - 1:
                        print("Retrying...")
                        time.sleep(5)  # Wait before retrying
                    else:
                        print("Maximum attempts reached. Publishing failed.")
                        # raise  # Optionally, re-raise the last exception


if __name__ == "__main__":
    chrome_debugging_port = "9222"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debugging_port}")
    driver = webdriver.Chrome(options=chrome_options)

    video_path = "videos/IMG_5303/IMG_5303_highlighted.mp4"
    thumbnail_path = "videos/IMG_5303/IMG_5303_cover.jpg"
    metadata_file_path = "videos/IMG_5303/IMG_5303_metadata.json"
    test_mode = True  # Set to False to disable test mode
    
    yt_publisher = YouTubePublisher(driver, video_path, thumbnail_path, metadata_file_path, test_mode)
    
    max_attempts = 1
    for attempt in range(max_attempts):
        try:
            yt_publisher.publish_video()
            break
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == max_attempts - 1:
                print("Maximum attempts reached. Publishing failed.")
            else:
                print("Retrying...")
                time.sleep(5)  # Wait before retrying
    
    # driver.quit()


Metadata loaded successfully.
Alert was present and dismissed.
Attached video videos/IMG_5303/IMG_5303_highlighted.mp4
The expected text is present in the span element.
The video title was set to "午餐困境：食不下嚥？ #午餐挑戰 #味覺考驗 #食物選擇 #生活困境 #味覺極限 #幽默日常 #飲食探險 #美食困惑 #日常生活 #食物評價"
The video description was set to "每個人的午餐時光都應該是一段愉快的時光，然而有時候它卻會變成一場味蕾的考驗。本影片展示了我們的主角在飢餓的驅動下，面對眼前令人卻步的食物。從食物的外觀到其味道，都向觀眾傳達出一種“看起來不好吃，聞起來也不好聞”的感覺。面對這樣的午餐，主角是否能夠咽下這頓飯？這段既幽默又引人思考的視頻將帶給觀眾深刻的觀看體驗。"
Thumbnail upload option not available.
Clicked on playlist dropdown
Selected playlist: SimpleLife
Clicked on the Done button.
Clicked on the Not Made for Kids button.
Clicked on the Show more button.
Tags entered: 午餐挑戰, 味覺考驗, 食物選擇, 生活困境, 味覺極限, 幽默日常, 飲食探險, 美食困惑, 日常生活, 食物評價,
Do you want to publish now? Type 'yes' to confirm: yes
Clicked on the Publish button.
Video published successfully.
